# 1. Setup Mario

In [1]:
# !pip install gym_super_mario_bros==7.3.0 nes_py
# !pip freeze

In [2]:
# Import the game
import gym
# Import the Joypad wrapper
from nes_py.wrappers import JoypadSpace
# Import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [3]:
# Setup game
env = gym.make('SuperMarioBros-1-1-v0', apply_api_compatibility=True, render_mode="human")
env = JoypadSpace(env, SIMPLE_MOVEMENT)

c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment SuperMarioBros-1-1-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\gym\envs\registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


In [5]:
# Create a flag - restart or not
done = True
# Loop through each frame in the game
for step in range(100): 
    # Start the game to begin with 
    if done: 
        # Start the gamee
        env.reset()
    # Do random actions
    state, reward, done, truncated, info = env.step(env.action_space.sample())
    # Show the game on the screen
    env.render()
# Close the game
env.close()

# 2. Preprocess Environment

In [6]:
# Install pytorch
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     ---------------------------------------- 2.4/2.4 GB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 3.2/3.2 MB 11.3 MB/s eta 0:00:00
     ------------------------------------- 336.6/336.6 kB 10.5 MB/s eta 0:00:00


In [9]:
# Install stable baselines for RL stuff
!pip install stable-baselines3

  Using cached stable_baselines3-1.8.0-py3-none-any.whl (174 kB)


In [12]:
# Import Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
import gym_super_mario_bros
# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Import Matplotlib to show the impact of frame stacking
from matplotlib import pyplot as plt

In [13]:
# 1. Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# 2. Simplify the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# 3. Grayscale
env = GrayScaleObservation(env, keep_dim=True)
# 4. Wrap inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# 5. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(


In [14]:
state = env.reset()

Unexpected exception formatting exception. Falling back to standard exception


c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
Traceback (most recent call last):
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Ruben\AppData\Local\Temp\ipykernel_10096\2599308756.py", line 1, in <module>
    state = env.reset()
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\stable_baselines3\common\vec_env\vec_frame_stack.py", line 38, in reset
    observation = self.venv.reset()  # pytype:disable=annotation-type-mismatch
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\stable_baselines3\common\vec_env\dummy_vec_env.py", line 74, in reset


In [17]:
state, reward, done, truncated, info = env.step([5])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Ruben\AppData\Local\Temp\ipykernel_10096\2981204275.py", line 1, in <module>
    state, reward, done, truncated, info = env.step([5])
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py", line 163, in step
    return self.step_wait()
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\stable_baselines3\common\vec_env\vec_frame_stack.py", line 33, in step_wait
    observations, rewards, dones, infos = self.venv.step_wait()
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\stable_baselines3\common\vec_env\dummy_vec_env.py", line 54, in step_wait
    obs, self.buf_rews[env_idx], self.buf_dones[env_idx], self.buf_infos[env_idx] = self.envs[env_idx].step(
  File "c:\Users\Ruben\miniconda3\envs\rl\

In [18]:
plt.figure(figsize=(20,16))
for idx in range(state.shape[3]):
    plt.subplot(1,4,idx+1)
    plt.imshow(state[0][:,:,idx])
plt.show()

IndexError: tuple index out of range

<Figure size 2000x1600 with 0 Axes>

# 3. Train the RL Model

In [19]:
# Import os for file path management
import os 
# Import PPO for algos
from stable_baselines3 import PPO
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [20]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [21]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [22]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [23]:
# This is the AI model started
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, 
            n_steps=512) 

Using cpu device
Wrapping the env in a VecTransposeImage.


In [24]:
# Train the AI model, this is where the AI model starts to learn
model.learn(total_timesteps=1000000, callback=callback)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Ruben\AppData\Local\Temp\ipykernel_10096\1897937656.py", line 2, in <module>
    model.learn(total_timesteps=1000000, callback=callback)
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\stable_baselines3\ppo\ppo.py", line 308, in learn
    return super().learn(
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py", line 239, in learn
    total_timesteps, callback = self._setup_learn(
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\stable_baselines3\common\base_class.py", line 412, in _setup_learn
    self._last_obs = self.env.reset()  # pytype: disable=annotation-type-mismatch
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\stable_baselines3\common\vec_env\vec_transpose.py", li

In [25]:
model.save('thisisatestmodel')

# 4. Test it Out

In [26]:
# Load model
model = PPO.load('./train/best_model_1000000')

FileNotFoundError: [Errno 2] No such file or directory: 'train\\best_model_1000000.zip'

In [27]:
state = env.reset()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Ruben\AppData\Local\Temp\ipykernel_10096\2599308756.py", line 1, in <module>
    state = env.reset()
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\stable_baselines3\common\vec_env\vec_frame_stack.py", line 38, in reset
    observation = self.venv.reset()  # pytype:disable=annotation-type-mismatch
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\stable_baselines3\common\vec_env\dummy_vec_env.py", line 74, in reset
    obs = self.envs[env_idx].reset()
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\gym\core.py", line 379, in reset
ValueError: too many values to unpack (expected 2)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\si

In [28]:
# Start the game 
state = env.reset()
# Loop through the game
while True: 
    
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Ruben\AppData\Local\Temp\ipykernel_10096\2993725579.py", line 2, in <module>
    state = env.reset()
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\stable_baselines3\common\vec_env\vec_frame_stack.py", line 38, in reset
    observation = self.venv.reset()  # pytype:disable=annotation-type-mismatch
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\stable_baselines3\common\vec_env\dummy_vec_env.py", line 74, in reset
    obs = self.envs[env_idx].reset()
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\site-packages\gym\core.py", line 379, in reset
ValueError: too many values to unpack (expected 2)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Ruben\miniconda3\envs\rl\lib\si